In [6]:
#need two new libraries
#pip install openpyxl
#pip install openmatrix  

In [1]:
import pandas as pd
import yaml
import plotly.io as pio
import os
pio.renderers.default = "plotly_mimetype+notebook_connected" 
from IPython.display import Markdown, display
import visualizer_helpers as vh
import openmatrix as omx
import gc #garbage collection? 
from pathlib import Path
import numpy as np 
import openpyxl
from openpyxl import load_workbook
import matplotlib.pyplot as plt

In [2]:
folder_path = r'C:\dev\asim_eet_viz-main\trip_tables'

In [101]:
# Define scenarios and methods
scenarios = ["base", "employment_scenario","transit_corridor_scenario"]
method_scenarios = ["MC", "EET"]
# Time periods and corresponding filenames
time_periods = ["AM", "EA", "EV", "MD", "PM"]
incomes = ["high", "med", "low"]

# Mode tab name patterns
sov_tabs = ["SOVNOTRPDR_{tp}", "SOVTRPDR_{tp}"]
sr_tabs = ["SR2NOTRPDR_{tp}", "SR2TRPDR_{tp}", "SR3NOTRPDR_{tp}", "SR3TRPDR_{tp}"]
walk_tabs = ["WALK_{tp}"]
bike_tabs = ["BIKE_{tp}"] 

# Initialize an empty dictionary to hold results by scenario
auto_by_scenario = {}
nonmotor_by_scenario = {}
transit_by_scenario = {}

In [75]:
#total trips by auto, sov, SR, walk, bike, transit
for scenario in scenarios:  
    print('Now processing scenario: ', scenario)
    for method in method_scenarios:
        print('Now processing method: ', method)
        # Result storage
        results = []
        nmot_results = [] 
        transit_results = [] 
        # read file directory
        files_in_directory = os.path.join(folder_path, scenario , method)  
        item_name = method+'_'+scenario
        print('Now processing method-scenario: ', item_name)
        folder = Path(files_in_directory)
        for tp in time_periods:
            for sc in incomes:
                filename = folder / f"autotrips_{tp}_{sc}.omx"
                with omx.open_file(filename, 'r') as f:
                    matrix_names = list(f.list_matrices())
        
                    # Auto (sum of all matrices)
                    auto_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                    total_auto = auto_sum.sum()
        
                    # SOV
                    sov_sum = sum(np.array(f[m.format(tp=tp)]) for m in sov_tabs if m.format(tp=tp) in matrix_names)
                    total_sov = sov_sum.sum()
        
                    # SR
                    sr_sum = sum(np.array(f[m.format(tp=tp)]) for m in sr_tabs if m.format(tp=tp) in matrix_names)
                    total_sr = sr_sum.sum()
        
                    results.append({
                        "time_period": tp,
                        "income": sc,
                        "auto": total_auto,
                        "SOV": total_sov,
                        "SR": total_sr
                    })
    
        # Convert to DataFrame
        summary_df = pd.DataFrame(results)  
        auto_by_scenario[item_name] = summary_df 
        print('Now printing: ', item_name, auto_by_scenario[item_name])  

        # now process non-motorized and transit trip tables 
        for tp in time_periods:
            file = folder / f"nmottrips_{tp}.omx"
            if not file.exists():
                print(f"File not found: {file}")
                continue
        
            with omx.open_file(file, 'r') as f: 
                matrix_names = list(f.list_matrices())
        
                # non-motorized (sum of all matrices)
                nonmotor_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                total_nonmotor = nonmotor_sum.sum()
        
                # walk
                walk_sum = sum(np.array(f[m.format(tp=tp)]) for m in walk_tabs if m.format(tp=tp) in matrix_names)
                total_walk = walk_sum.sum()
                
                # bike
                bike_sum = sum(np.array(f[m.format(tp=tp)]) for m in bike_tabs if m.format(tp=tp) in matrix_names)
                total_bike = bike_sum.sum() 
        
                nmot_results.append({
                    "time_period": tp,
                    "non-motorized": total_nonmotor,
                    "bike": total_bike,
                    "walk": total_walk
                }) 
                
            tranfile = folder / f"trantrips_{tp}.omx"
            if not tranfile.exists():
                print(f"File not found: {tranfile}")
                continue
        
            with omx.open_file(tranfile, 'r') as f: 
                matrix_names = list(f.list_matrices())
        
                # transit (sum of all matrices)
                transit_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                total_transit = transit_sum.sum() 
        
                transit_results.append({
                    "time_period": tp, 
                    "transit": total_transit 
                })
        
        # Convert to DataFrame
        nmot_summary_df = pd.DataFrame(nmot_results)
        nonmotor_by_scenario[item_name] = nmot_summary_df
        transit_summary_df = pd.DataFrame(transit_results)
        transit_by_scenario[item_name] = transit_summary_df
        # Show the result
        print('Now printing: ', item_name, nonmotor_by_scenario[item_name])  
        print('Now printing: ', item_name, transit_by_scenario[item_name])  

Now processing scenario:  base
Now processing method:  MC
Now processing method-scenario:  MC_base
Now printing:  MC_base    time_period income          auto       SOV             SR
0           AM   high  6.269150e+05  363993.0  262921.968468
1           AM    med  6.405752e+05  483110.0  157465.241742
2           AM    low  5.552257e+05  466128.0   89097.681682
3           EA   high  4.196994e+04   34017.0    7952.941441
4           EA    med  4.855385e+04   44749.0    3804.846847
5           EA    low  4.642801e+04   44656.0    1772.010511
6           EV   high  3.476257e+05  213632.0  133993.738739
7           EV    med  3.548584e+05  295187.0   59671.366366
8           EV    low  3.173658e+05  292286.0   25079.783784
9           MD   high  1.128140e+06  769257.0  358882.543544
10          MD    med  1.125935e+06  951928.0  174007.466967
11          MD    low  1.022751e+06  937441.0   85309.698198
12          PM   high  7.512506e+05  459830.0  291420.578078
13          PM    med  7

## Export to Excel (total trips)

In [3]:
# Define the output folder path 
output_folder = "output"  

# Check if the folder exists, create it if not
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created folder: {output_folder}")
else:
    print(f"Folder already exists: {output_folder}")

Folder already exists: output


In [80]:
# Export summaries to excel template
# Load the workbook  
with pd.ExcelWriter("output/template.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    auto_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base" , startcol=0, index=True)
    nonmotor_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base",  startcol=10, index=False)
    transit_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base",  startcol=18, index=False)
    auto_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base" , startcol=0, index=True)
    nonmotor_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base",  startcol=10, index=False)
    transit_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base",  startcol=18, index=False)
    auto_by_scenario['MC_employment_scenario'].to_excel(writer, sheet_name="MC_employment" , startcol=0, index=True)
    nonmotor_by_scenario['MC_employment_scenario'].to_excel(writer, sheet_name="MC_employment",  startcol=10, index=False)
    transit_by_scenario['MC_employment_scenario'].to_excel(writer, sheet_name="MC_employment",  startcol=18, index=False)
    auto_by_scenario['EET_employment_scenario'].to_excel(writer, sheet_name="EET_employment" , startcol=0, index=True)
    nonmotor_by_scenario['EET_employment_scenario'].to_excel(writer, sheet_name="EET_employment",  startcol=10, index=False)
    transit_by_scenario['EET_employment_scenario'].to_excel(writer, sheet_name="EET_employment",  startcol=18, index=False)
    auto_by_scenario['MC_transit_corridor_scenario'].to_excel(writer, sheet_name="MC_transit_corridor" , startcol=0, index=True)
    nonmotor_by_scenario['MC_transit_corridor_scenario'].to_excel(writer, sheet_name="MC_transit_corridor",  startcol=10, index=False)
    transit_by_scenario['MC_transit_corridor_scenario'].to_excel(writer, sheet_name="MC_transit_corridor",  startcol=18, index=False)
    auto_by_scenario['EET_transit_corridor_scenario'].to_excel(writer, sheet_name="EET_transit_corridor" , startcol=0, index=True)
    nonmotor_by_scenario['EET_transit_corridor_scenario'].to_excel(writer, sheet_name="EET_transit_corridor",  startcol=10, index=False)
    transit_by_scenario['EET_transit_corridor_scenario'].to_excel(writer, sheet_name="EET_transit_corridor",  startcol=18, index=False)
    


## calculate max diff and RMSE %

In [134]:
def percent_rmse(observed, predicted):
    """Calculate percent RMSE, using matrices"""
    observed = np.array(observed)
    predicted = np.array(predicted)

    rmse = np.sqrt(((predicted- observed) ** 2).mean())
    base_sum = observed.sum()
    base_mean = observed.mean()
    return rmse, base_sum, base_mean


In [158]:
def sum_all_matrices(file_path):
    with omx.open_file(file_path, 'r') as f:
        matrix_names = f.list_matrices()
        #print(f"{file_path.name} contains matrices: {matrix_names}")

        # Start with a zero matrix of the correct shape
        shape = f[matrix_names[0]].shape
        total = np.zeros(shape)

        for name in matrix_names:
            total += np.array(f[name])

    return total

In [146]:
scenarios = ["employment_scenario","transit_corridor_scenario"]
methods = ["MC", "EET"]
for method in methods:
    for scenario in scenarios:   
        base_directory = os.path.join(folder_path, 'base', method)  
        scenario_directory = os.path.join(folder_path, scenario , method)   
        base_folder = Path(base_directory)
        scenario_folder = Path(scenario_directory)
        
        for tp in time_periods:
            for sc in incomes:
                print('Now processing method: ', method) 
                print('Now processing scenario: ', scenario) 
                print(tp, ' by ', sc)
                base_file = base_folder / f"autotrips_{tp}_{sc}.omx"
                scenario_file = scenario_folder / f"autotrips_{tp}_{sc}.omx" 
                base_total = sum_all_matrices(base_file)
                scenario_total = sum_all_matrices(scenario_file)  
                # Maximum absolute difference (magnitude)
                diff_total = scenario_total - base_total
                max_abs_diff = np.max(np.abs(diff_total))
                print(f"Maximum absolute difference in any cell: {max_abs_diff:.2f} trips")
                rmse = np.sqrt(np.mean((scenario_total - base_total) ** 2)) 
                print(f"RMSE between summed matrices: {rmse:.2f} trips") 
                normalized_rmse = rmse / base_total.mean()
                print(f"Average of the base matrix: {base_total.mean():.6f}") 
                print(f"Normalized rmse: {normalized_rmse:.2f}") 
                percent_rmse = 100 * normalized_rmse
                print(f"% RMSE: {percent_rmse:.2f}%") 

Now processing method:  MC
Now processing scenario:  employment_scenario
AM  by  high
Maximum absolute difference in any cell: 83.60 trips
RMSE between summed matrices: 0.18 trips
Average of the base matrix: 0.025617
Normalized rmse: 7.03
% RMSE: 703.25%
Now processing method:  MC
Now processing scenario:  employment_scenario
AM  by  med
Maximum absolute difference in any cell: 73.30 trips
RMSE between summed matrices: 0.19 trips
Average of the base matrix: 0.026175
Normalized rmse: 7.36
% RMSE: 735.65%
Now processing method:  MC
Now processing scenario:  employment_scenario
AM  by  low
Maximum absolute difference in any cell: 54.70 trips
RMSE between summed matrices: 0.18 trips
Average of the base matrix: 0.022687
Normalized rmse: 8.10
% RMSE: 810.43%
Now processing method:  MC
Now processing scenario:  employment_scenario
EA  by  high
Maximum absolute difference in any cell: 6.30 trips
RMSE between summed matrices: 0.05 trips
Average of the base matrix: 0.001715
Normalized rmse: 31.3

In [148]:
scenarios = ["employment_scenario","transit_corridor_scenario"]
methods = ["MC", "EET"]
for method in methods:
    for scenario in scenarios:   
        base_directory = os.path.join(folder_path, 'base', method)  
        scenario_directory = os.path.join(folder_path, scenario , method)   
        base_folder = Path(base_directory)
        scenario_folder = Path(scenario_directory)
        
        for tp in time_periods: 
            print('Now processing method: ', method) 
            print('Now processing scenario: ', scenario) 
            print("Transit Trip by ", tp)
            base_file = base_folder / f"trantrips_{tp}.omx"
            scenario_file = scenario_folder / f"trantrips_{tp}.omx" 
            base_total = sum_all_matrices(base_file)
            scenario_total = sum_all_matrices(scenario_file)  
            # Maximum absolute difference (magnitude)
            diff_total = scenario_total - base_total
            max_abs_diff = np.max(np.abs(diff_total))
            print(f"Maximum absolute difference in any cell: {max_abs_diff:.2f} trips")
            rmse = np.sqrt(np.mean((scenario_total - base_total) ** 2)) 
            print(f"RMSE between summed matrices: {rmse:.2f} trips") 
            normalized_rmse = rmse / base_total.mean()
            print(f"Average of the base matrix: {base_total.mean():.6f}") 
            print(f"Normalized rmse: {normalized_rmse:.2f}") 
            percent_rmse = 100 * normalized_rmse
            print(f"% RMSE: {percent_rmse:.2f}%") 

            print("Non-motorized Trip by ", tp)
            base_file = base_folder / f"nmottrips_{tp}.omx"
            scenario_file = scenario_folder / f"nmottrips_{tp}.omx" 
            base_total = sum_all_matrices(base_file)
            scenario_total = sum_all_matrices(scenario_file)  
            # Maximum absolute difference (magnitude)
            diff_total = scenario_total - base_total
            max_abs_diff = np.max(np.abs(diff_total))
            print(f"Maximum absolute difference in any cell: {max_abs_diff:.2f} trips")
            rmse = np.sqrt(np.mean((scenario_total - base_total) ** 2)) 
            print(f"RMSE between summed matrices: {rmse:.2f} trips") 
            normalized_rmse = rmse / base_total.mean()
            print(f"Average of the base matrix: {base_total.mean():.6f}") 
            print(f"Normalized rmse: {normalized_rmse:.2f}") 
            percent_rmse = 100 * normalized_rmse
            print(f"% RMSE: {percent_rmse:.2f}%")
            

Now processing method:  MC
Now processing scenario:  employment_scenario
Transit Trip by  AM
Maximum absolute difference in any cell: 7.00 trips
RMSE between summed matrices: 0.05 trips
Average of the base matrix: 0.001207
Normalized rmse: 38.23
% RMSE: 3822.79%
Non-motorized Trip by  AM
Maximum absolute difference in any cell: 114.00 trips
RMSE between summed matrices: 0.15 trips
Average of the base matrix: 0.018568
Normalized rmse: 7.95
% RMSE: 795.48%
Now processing method:  MC
Now processing scenario:  employment_scenario
Transit Trip by  EA
Maximum absolute difference in any cell: 7.00 trips
RMSE between summed matrices: 0.02 trips
Average of the base matrix: 0.000127
Normalized rmse: 123.76
% RMSE: 12376.13%
Non-motorized Trip by  EA
Maximum absolute difference in any cell: 15.00 trips
RMSE between summed matrices: 0.04 trips
Average of the base matrix: 0.000759
Normalized rmse: 47.00
% RMSE: 4700.34%
Now processing method:  MC
Now processing scenario:  employment_scenario
Transi

## 

## Additional RMSE to discount small values

In [172]:
# Number of cells in the matrix that change from less than 1 trip (0 to 1) in the Base to at least 10 trips in the Scenario
# Number of cells in the matrix that change from at least 10 trips in the Base to less than 1 trip (0 to 1) in the Scenario
# %RMSE for cells in which either the Base or the Scenario has a minimum of 10 trips (also provide number of cells that meet this criteria)
# %RMSE for cells in which either the Base or the Scenario has a minimum of 50 trips (also provide number of cells that meet this criteria)

In [167]:
def percent_rmse_with_min_threshold(base_matrix, scenario_matrix, threshold=10):
    """
    Compute %RMSE and number of cells where either the base or scenario
    matrix has values ≥ threshold.

    Parameters:
        base_matrix (np.ndarray): Base trip matrix.
        scenario_matrix (np.ndarray): Scenario trip matrix.
        threshold (float): Minimum trip count threshold (default: 10).

    Returns:
        dict: Contains RMSE, mean of base, %RMSE, and number of cells used.
    """
    if base_matrix.shape != scenario_matrix.shape:
        raise ValueError("Base and scenario matrices must have the same shape.")

    # Identify relevant cells
    mask = (base_matrix >= threshold) | (scenario_matrix >= threshold)

    # If no matching cells, return None safely
    if not np.any(mask):
        return {
            "num_cells": 0,
            "rmse": None,
            "mean_base": None,
            "percent_rmse": None
        }

    # Compute RMSE
    squared_diff = (base_matrix - scenario_matrix) ** 2
    rmse = np.sqrt(np.mean(squared_diff[mask]))

    # Compute average of base in those cells
    mean_base = base_matrix[mask].mean()

    # Compute %RMSE
    percent_rmse = (rmse / mean_base) * 100

    return {
        "num_cells": np.count_nonzero(mask),
        "rmse": rmse,
        "mean_base": mean_base,
        "percent_rmse": percent_rmse
    }


In [170]:
def summarize_threshold_rmse(base_total, scenario_total, count1, count2, thresholds=(10, 50)):
    """
    Print a detailed summary of RMSE metrics for given thresholds and change counts.
    
    Parameters:
        base_total (np.ndarray): Base matrix.
        scenario_total (np.ndarray): Scenario matrix.
        count1 (int): Number of cells that changed from <1 to ≥10 trips.
        count2 (int): Number of cells that changed from ≥10 to <1 trips.
        thresholds (tuple): Threshold values for RMSE computation (default: (10, 50)).
    """
    print(f"Number of cells that changed from <1 to ≥10 trips: {count1}")
    print(f"Number of cells that changed from ≥10 to <1 trips: {count2}")
    
    for threshold in thresholds:
        results = percent_rmse_with_min_threshold(base_total, scenario_total, threshold=threshold)
        label = f"{threshold} trips"
        if results["num_cells"] > 0 and results["rmse"] is not None:
            print(f"\n--- For threshold ≥ {label} ---")
            print(f"Number of cells: {results['num_cells']}")
            print(f"RMSE: {results['rmse']:.4f}")
            print(f"% RMSE: {results['percent_rmse']:.2f}%")
        else:
            print(f"\n--- For threshold ≥ {label} ---")
            print("No cells met the threshold condition; RMSE cannot be computed.")


In [173]:
scenarios = ["employment_scenario","transit_corridor_scenario"]
methods = ["MC", "EET"] 

for method in methods:
    for scenario in scenarios:   
        base_directory = os.path.join(folder_path, 'base', method)  
        scenario_directory = os.path.join(folder_path, scenario , method)   
        base_folder = Path(base_directory)
        scenario_folder = Path(scenario_directory)
        
        for tp in time_periods:
            for sc in incomes:
                print(f"\nNow processing method: {method}  |  Scenario: {scenario}")
                print(f"Auto trips - Time Period: {tp}  |  Income Level: {sc}\n")  
                base_file = base_folder / f"autotrips_{tp}_{sc}.omx"
                scenario_file = scenario_folder / f"autotrips_{tp}_{sc}.omx" 
                base_total = sum_all_matrices(base_file)
                scenario_total = sum_all_matrices(scenario_file)   

                # Define condition: from [0 to <1] in base to >=10 in scenario
                condition1 = (base_total < 1) & (scenario_total >= 10)
                condition2 = (base_total >=10) & (scenario_total < 10)
                # Count how many cells meet the condition
                count1 = np.count_nonzero(condition1)
                count2 = np.count_nonzero(condition2)
                #RMSE
                summarize_threshold_rmse(base_total, scenario_total, count1, count2)

        for tp in time_periods:  
            # transit 
            print(f"\nNow processing method: {method}  |  Scenario: {scenario}")
            print(f"Transit trips - Time Period: {tp}") 
            base_file = base_folder / f"trantrips_{tp}.omx"
            scenario_file = scenario_folder / f"trantrips_{tp}.omx" 
            base_total = sum_all_matrices(base_file)
            scenario_total = sum_all_matrices(scenario_file)    
            # conditions
            condition1 = (base_total < 1) & (scenario_total >= 10)
            condition2 = (base_total >=10) & (scenario_total < 10)
            count1 = np.count_nonzero(condition1)
            count2 = np.count_nonzero(condition2)
            summarize_threshold_rmse(base_total, scenario_total, count1, count2) 
            # walk/bile
            print(f"\nNow processing method: {method}  |  Scenario: {scenario}")
            print(f"Non-motorized trips - Time Period: {tp}") 
            base_file = base_folder / f"nmottrips_{tp}.omx"
            scenario_file = scenario_folder / f"nmottrips_{tp}.omx" 
            base_total = sum_all_matrices(base_file)
            scenario_total = sum_all_matrices(scenario_file)   
            
            # condition
            count1 = np.count_nonzero(condition1)
            count2 = np.count_nonzero(condition2)
            summarize_threshold_rmse(base_total, scenario_total, count1, count2)


Now processing method: MC  |  Scenario: employment_scenario
Auto trips - Time Period: AM  |  Income Level: high

Number of cells that changed from <1 to ≥10 trips: 0
Number of cells that changed from ≥10 to <1 trips: 504

--- For threshold ≥ 10 trips ---
Number of cells: 3261
RMSE: 4.5735
% RMSE: 23.81%

--- For threshold ≥ 50 trips ---
Number of cells: 156
RMSE: 10.7977
% RMSE: 11.64%

Now processing method: MC  |  Scenario: employment_scenario
Auto trips - Time Period: AM  |  Income Level: med

Number of cells that changed from <1 to ≥10 trips: 4
Number of cells that changed from ≥10 to <1 trips: 483

--- For threshold ≥ 10 trips ---
Number of cells: 2800
RMSE: 4.6452
% RMSE: 25.44%

--- For threshold ≥ 50 trips ---
Number of cells: 114
RMSE: 10.6722
% RMSE: 11.49%

Now processing method: MC  |  Scenario: employment_scenario
Auto trips - Time Period: AM  |  Income Level: low

Number of cells that changed from <1 to ≥10 trips: 1
Number of cells that changed from ≥10 to <1 trips: 390


## Affected Zones - Employment Scenario

In [4]:
def sum_trips_from_or_to_affected_zones(matrix, taz_csv_path, taz_column='taz', is_1_based=False):
    """
    Sum trips in a matrix where either origin or destination is in the affected TAZ list.
    
    Parameters:
        matrix (np.ndarray): OD trip matrix (square 2D array).
        taz_csv_path (str or Path): Path to CSV file with a column of TAZs.
        taz_column (str): Column name in the CSV that contains TAZ IDs.
        is_1_based (bool): If True, TAZs are treated as 1-based and will be adjusted to 0-based.

    Returns:
        total_trips (float): Sum of trips where origin or destination is in the TAZ list.
    """
    # Load and deduplicate TAZ list
    df = pd.read_csv(taz_csv_path)
    affected_tazs = np.unique(df[taz_column].values)

    # Adjust if TAZs are 1-based
    if is_1_based:
        affected_tazs = affected_tazs - 1

    # Sanity check
    n_zones = matrix.shape[0]
    assert matrix.shape[0] == matrix.shape[1], "Matrix must be square."
    assert affected_tazs.max() < n_zones, "TAZ index out of bounds."

    # Create masks for origin and destination
    origin_mask = np.zeros((n_zones, n_zones), dtype=bool)
    destination_mask = np.zeros((n_zones, n_zones), dtype=bool)
    origin_mask[affected_tazs, :] = True
    destination_mask[:, affected_tazs] = True

    # Combine without double-counting
    combined_mask = origin_mask | destination_mask
    total_trips = matrix[combined_mask].sum()

    return total_trips

def sum_trips_within_affected_zones(matrix, taz_csv_path, taz_column='taz', is_1_based=False):
    """
    Sum trips in a matrix where both origin and destination are in the affected TAZ list.
    
    Parameters:
        matrix (np.ndarray): OD trip matrix (square 2D array).
        taz_csv_path (str or Path): Path to CSV file with a column of TAZs.
        taz_column (str): Column name in the CSV that contains TAZ IDs.
        is_1_based (bool): If True, TAZs are treated as 1-based and will be adjusted to 0-based.

    Returns:
        total_trips (float): Sum of trips where both origin and destination are in the TAZ list.
    """
    # Load and deduplicate TAZ list
    df = pd.read_csv(taz_csv_path)
    affected_tazs = np.unique(df[taz_column].values)

    # Adjust if TAZs are 1-based
    if is_1_based:
        affected_tazs = affected_tazs - 1

    # Sanity checks
    n_zones = matrix.shape[0]
    assert matrix.shape[0] == matrix.shape[1], "Matrix must be square."
    assert affected_tazs.max() < n_zones, "TAZ index out of bounds."

    # Use np.ix_ to slice submatrix where both O and D are affected
    submatrix = matrix[np.ix_(affected_tazs, affected_tazs)]
    total_trips = submatrix.sum()

    return total_trips

In [161]:
taz_affected = Path(r"C:\dev\asim_eet_viz-main\input\employment_zones_affected.csv")
#taz_transit = Path(r"C:\dev\asim_eet_viz-main\input\transit_zones_affected.csv") 
# Employment scenarios
scenarios = ["base", "employment_scenario"] 
method_scenarios = ["MC", "EET"]
# Time periods and corresponding filenames
time_periods = ["AM", "EA", "EV", "MD", "PM"]
incomes = ["high", "med", "low"]

# Mode tab name patterns
sov_tabs = ["SOVNOTRPDR_{tp}", "SOVTRPDR_{tp}"]
sr_tabs = ["SR2NOTRPDR_{tp}", "SR2TRPDR_{tp}", "SR3NOTRPDR_{tp}", "SR3TRPDR_{tp}"]
walk_tabs = ["WALK_{tp}"]
bike_tabs = ["BIKE_{tp}"] 

# Initialize an empty dictionary to hold results by scenario
auto_by_scenario = {}
nonmotor_by_scenario = {}
transit_by_scenario = {}

In [162]:
#total trips by auto, sov, SR, walk, bike, transit
for scenario in scenarios:  
    print('Now processing scenario: ', scenario)
    for method in method_scenarios:
        print('Now processing method: ', method)
        # Result storage
        results = []
        nmot_results = [] 
        transit_results = [] 
        # read file directory
        files_in_directory = os.path.join(folder_path, scenario , method)  
        item_name = method+'_'+scenario
        print('Now processing method-scenario: ', item_name)
        folder = Path(files_in_directory)
        for tp in time_periods:
            for sc in incomes:
                filename = folder / f"autotrips_{tp}_{sc}.omx"
                with omx.open_file(filename, 'r') as f:
                    matrix_names = list(f.list_matrices())
        
                    # Auto (sum of all matrices)
                    auto_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                    total_auto = sum_trips_from_or_to_affected_zones(matrix=auto_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
                     
                    # SOV
                    sov_sum = sum(np.array(f[m.format(tp=tp)]) for m in sov_tabs if m.format(tp=tp) in matrix_names)
                    total_sov = sum_trips_from_or_to_affected_zones(matrix=sov_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                    # SR
                    sr_sum = sum(np.array(f[m.format(tp=tp)]) for m in sr_tabs if m.format(tp=tp) in matrix_names)
                    total_sr = sum_trips_from_or_to_affected_zones(matrix=sr_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                    results.append({
                        "time_period": tp,
                        "income": sc,
                        "auto": total_auto,
                        "SOV": total_sov,
                        "SR": total_sr
                    })
    
        # Convert to DataFrame
        summary_df = pd.DataFrame(results)  
        auto_by_scenario[item_name] = summary_df 
        print('Now printing: ', item_name, auto_by_scenario[item_name])  

        # now process non-motorized and transit trip tables 
        for tp in time_periods:
            file = folder / f"nmottrips_{tp}.omx"
            if not file.exists():
                print(f"File not found: {file}")
                continue
        
            with omx.open_file(file, 'r') as f: 
                matrix_names = list(f.list_matrices())
        
                # non-motorized (sum of all matrices)
                nonmotor_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                total_nonmotor = sum_trips_from_or_to_affected_zones(matrix=nonmotor_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                # walk
                walk_sum = sum(np.array(f[m.format(tp=tp)]) for m in walk_tabs if m.format(tp=tp) in matrix_names)
                total_walk =  sum_trips_from_or_to_affected_zones(matrix=walk_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
                
                # bike
                bike_sum = sum(np.array(f[m.format(tp=tp)]) for m in bike_tabs if m.format(tp=tp) in matrix_names)
                total_bike = sum_trips_from_or_to_affected_zones(matrix=bike_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                nmot_results.append({
                    "time_period": tp,
                    "non-motorized": total_nonmotor,
                    "bike": total_bike,
                    "walk": total_walk
                }) 
                
            tranfile = folder / f"trantrips_{tp}.omx"
            if not tranfile.exists():
                print(f"File not found: {tranfile}")
                continue
        
            with omx.open_file(tranfile, 'r') as f: 
                matrix_names = list(f.list_matrices())
        
                # transit (sum of all matrices)
                transit_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                total_transit = sum_trips_from_or_to_affected_zones(matrix=transit_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                transit_results.append({
                    "time_period": tp, 
                    "transit": total_transit 
                })
        
        # Convert to DataFrame
        nmot_summary_df = pd.DataFrame(nmot_results)
        nonmotor_by_scenario[item_name] = nmot_summary_df
        transit_summary_df = pd.DataFrame(transit_results)
        transit_by_scenario[item_name] = transit_summary_df
        # Show the result
        print('Now printing: ', item_name, nonmotor_by_scenario[item_name])  
        print('Now printing: ', item_name, transit_by_scenario[item_name])  

Now processing scenario:  base
Now processing method:  MC
Now processing method-scenario:  MC_base
Now printing:  MC_base    time_period income          auto      SOV            SR
0           AM   high  36982.885886  24130.0  12852.885886
1           AM    med  37703.469970  30596.0   7107.469970
2           AM    low  32543.133634  28571.0   3972.133634
3           EA   high   2752.325826   2238.0    514.325826
4           EA    med   3070.944444   2885.0    185.944444
5           EA    low   2736.418919   2646.0     90.418919
6           EV   high  24465.816817  15421.0   9044.816817
7           EV    med  23888.800300  20343.0   3545.800300
8           EV    low  20179.330330  18739.0   1440.330330
9           MD   high  88881.674174  64093.0  24788.674174
10          MD    med  82842.725225  72071.0  10771.725225
11          MD    low  71647.674174  66642.0   5005.674174
12          PM   high  50159.255255  33093.0  17066.255255
13          PM    med  51239.439940  42743.0   8496.

In [163]:
# Export summaries to excel template
# Load the workbook  
with pd.ExcelWriter("output/MC_EET_Summaries.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    auto_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base" ,  startcol=25, index=True)
    nonmotor_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base",   startcol=35, index=False)
    transit_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base",   startcol=40, index=False)
    auto_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base" ,  startcol=25, index=True)
    nonmotor_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base",    startcol=35, index=False)
    transit_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base",    startcol=40, index=False)
    auto_by_scenario['MC_employment_scenario'].to_excel(writer, sheet_name="MC_employment" ,   startcol=25, index=True)
    nonmotor_by_scenario['MC_employment_scenario'].to_excel(writer, sheet_name="MC_employment",    startcol=35, index=False)
    transit_by_scenario['MC_employment_scenario'].to_excel(writer, sheet_name="MC_employment",    startcol=40, index=False)
    auto_by_scenario['EET_employment_scenario'].to_excel(writer, sheet_name="EET_employment" ,  startcol=25, index=True)
    nonmotor_by_scenario['EET_employment_scenario'].to_excel(writer, sheet_name="EET_employment",    startcol=35, index=False)
    transit_by_scenario['EET_employment_scenario'].to_excel(writer, sheet_name="EET_employment",    startcol=40, index=False) 

## Affected Zones - Transit Corridor Scenario 

In [7]:
#Transit scenario
#taz_affected = Path(r"C:\dev\asim_eet_viz-main\input\employment_zones_affected.csv")
taz_affected = Path(r"C:\dev\asim_eet_viz-main\input\transit_zones_affected.csv") 
scenarios = ["base", "transit_corridor_scenario"] 
method_scenarios = ["MC", "EET"]
# Time periods and corresponding filenames
time_periods = ["AM", "EA", "EV", "MD", "PM"]
incomes = ["high", "med", "low"]

# Mode tab name patterns
sov_tabs = ["SOVNOTRPDR_{tp}", "SOVTRPDR_{tp}"]
sr_tabs = ["SR2NOTRPDR_{tp}", "SR2TRPDR_{tp}", "SR3NOTRPDR_{tp}", "SR3TRPDR_{tp}"]
walk_tabs = ["WALK_{tp}"]
bike_tabs = ["BIKE_{tp}"] 

# Initialize an empty dictionary to hold results by scenario
auto_by_scenario = {}
nonmotor_by_scenario = {}
transit_by_scenario = {}

In [ ]:
#total trips by auto, sov, SR, walk, bike, transit
for scenario in scenarios:  
    print('Now processing scenario: ', scenario)
    for method in method_scenarios:
        print('Now processing method: ', method)
        # Result storage
        results = []
        nmot_results = [] 
        transit_results = [] 
        # read file directory
        files_in_directory = os.path.join(folder_path, scenario , method)  
        item_name = method+'_'+scenario
        print('Now processing method-scenario: ', item_name)
        folder = Path(files_in_directory)
        for tp in time_periods:
            for sc in incomes:
                filename = folder / f"autotrips_{tp}_{sc}.omx"
                with omx.open_file(filename, 'r') as f:
                    matrix_names = list(f.list_matrices())
        
                    # Auto (sum of all matrices)
                    auto_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                    total_auto = sum_trips_within_affected_zones(matrix=auto_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
                    
        
                    # SOV
                    sov_sum = sum(np.array(f[m.format(tp=tp)]) for m in sov_tabs if m.format(tp=tp) in matrix_names)
                    total_sov = sum_trips_within_affected_zones(matrix=sov_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                    # SR
                    sr_sum = sum(np.array(f[m.format(tp=tp)]) for m in sr_tabs if m.format(tp=tp) in matrix_names)
                    total_sr = sum_trips_within_affected_zones(matrix=sr_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                    results.append({
                        "time_period": tp,
                        "income": sc,
                        "auto": total_auto,
                        "SOV": total_sov,
                        "SR": total_sr
                    })
    
        # Convert to DataFrame
        summary_df = pd.DataFrame(results)  
        auto_by_scenario[item_name] = summary_df
        print('Now printing: ', item_name, auto_by_scenario[item_name])  

        # now process non-motorized and transit trip tables 
        for tp in time_periods:
            file = folder / f"nmottrips_{tp}.omx"
            if not file.exists():
                print(f"File not found: {file}")
                continue
        
            with omx.open_file(file, 'r') as f: 
                matrix_names = list(f.list_matrices())
        
                # non-motorized (sum of all matrices)
                nonmotor_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                total_nonmotor = sum_trips_within_affected_zones(matrix=nonmotor_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                # walk
                walk_sum = sum(np.array(f[m.format(tp=tp)]) for m in walk_tabs if m.format(tp=tp) in matrix_names)
                total_walk =  sum_trips_within_affected_zones(matrix=walk_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
                
                # bike
                bike_sum = sum(np.array(f[m.format(tp=tp)]) for m in bike_tabs if m.format(tp=tp) in matrix_names)
                total_bike = sum_trips_within_affected_zones(matrix=bike_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                nmot_results.append({
                    "time_period": tp,
                    "non-motorized": total_nonmotor,
                    "bike": total_bike,
                    "walk": total_walk
                }) 
                
            tranfile = folder / f"trantrips_{tp}.omx"
            if not tranfile.exists():
                print(f"File not found: {tranfile}")
                continue
        
            with omx.open_file(tranfile, 'r') as f: 
                matrix_names = list(f.list_matrices())
        
                # transit (sum of all matrices)
                transit_sum = sum(np.array(f[matrix]) for matrix in matrix_names)
                total_transit = sum_trips_within_affected_zones(matrix=transit_sum,taz_csv_path=taz_affected, taz_column='taz',  is_1_based=True)
        
                transit_results.append({
                    "time_period": tp, 
                    "transit": total_transit 
                })
        
        # Convert to DataFrame
        nmot_summary_df = pd.DataFrame(nmot_results)
        nonmotor_by_scenario[item_name] = nmot_summary_df
        transit_summary_df = pd.DataFrame(transit_results)
        transit_by_scenario[item_name] = transit_summary_df
        # Show the result
        print('Now printing: ', item_name, nonmotor_by_scenario[item_name])  
        print('Now printing: ', item_name, transit_by_scenario[item_name])  

Now processing scenario:  base
Now processing method:  MC
Now processing method-scenario:  MC_base
Now printing:  MC_base    time_period income          auto      SOV            SR
0           AM   high  33197.918919  17865.0  15332.918919
1           AM    med  30465.006006  22098.0   8367.006006
2           AM    low  25248.408408  20749.0   4499.408408
3           EA   high   1843.645646   1499.0    344.645646
4           EA    med   1942.930931   1809.0    133.930931
5           EA    low   1874.213213   1797.0     77.213213
6           EV   high  16600.629129  10203.0   6397.629129
7           EV    med  15812.512012  13145.0   2667.512012
8           EV    low  13842.358859  12714.0   1128.358859
9           MD   high  62990.147147  44119.0  18871.147147
10          MD    med  53486.240240  45221.0   8265.240240
11          MD    low  45238.797297  41179.0   4059.797297
12          PM   high  38540.603604  22386.0  16154.603604
13          PM    med  35694.810811  27646.0   8048.

In [166]:
# Export summaries to excel template
# Load the workbook  
with pd.ExcelWriter("output/MC_EET_Summaries.xlsx", engine="openpyxl", mode="a", if_sheet_exists="overlay") as writer:
    auto_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base" ,  startcol=50, index=True)
    nonmotor_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base",   startcol=60, index=False)
    transit_by_scenario['MC_base'].to_excel(writer, sheet_name="MC_base",   startcol=68, index=False)
    auto_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base" ,  startcol=50, index=True)
    nonmotor_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base",    startcol=60, index=False)
    transit_by_scenario['EET_base'].to_excel(writer, sheet_name="EET_base",    startcol=68, index=False) 
    auto_by_scenario['MC_transit_corridor_scenario'].to_excel(writer, sheet_name="MC_transit_corridor" , startcol=50, index=True)
    nonmotor_by_scenario['MC_transit_corridor_scenario'].to_excel(writer, sheet_name="MC_transit_corridor",  startcol=60, index=False)
    transit_by_scenario['MC_transit_corridor_scenario'].to_excel(writer, sheet_name="MC_transit_corridor",  startcol=68, index=False)
    auto_by_scenario['EET_transit_corridor_scenario'].to_excel(writer, sheet_name="EET_transit_corridor" , startcol=50, index=True)
    nonmotor_by_scenario['EET_transit_corridor_scenario'].to_excel(writer, sheet_name="EET_transit_corridor",  startcol=60, index=False)
    transit_by_scenario['EET_transit_corridor_scenario'].to_excel(writer, sheet_name="EET_transit_corridor",  startcol=68, index=False)
    
